<a href="https://colab.research.google.com/github/Malik-Raheel/data_analysis/blob/main/strong_entries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas_ta
!pip install ccxt
!pip install pandas
!pip install numpy
!pip install ta
!pip install scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=d6dfd5632f09a18dd7df287bf821f2a18284cd9b0b0b2db92a265ce9cf44ec95
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl s

In [4]:
import ccxt
import pandas as pd
import numpy as np
import ta
from scipy.stats import gaussian_kde

# Initialize KuCoin API
exchange = ccxt.kucoin({
    'rateLimit': 1200,
    'enableRateLimit': True,
})


# Fetch historical data
def fetch_data(symbol, timeframe, limit=500):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    return data

# Calculate Hurst Exponent
def hurst_exponent(series, lag_max=20):
    lags = range(2, lag_max)
    variances = [np.std(np.diff(series, lag)) for lag in lags]
    log_variances = np.log(variances)
    log_lags = np.log(lags)
    slope = np.polyfit(log_lags, log_variances, 1)[0]
    return slope * 2

# Calculate Indicators
def calculate_indicators(df):
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    df['EMA20'] = ta.trend.EMAIndicator(df['close'], window=20).ema_indicator()
    df['EMA50'] = ta.trend.EMAIndicator(df['close'], window=50).ema_indicator()
    df['EMA200'] = ta.trend.EMAIndicator(df['close'], window=200).ema_indicator()
    df['ATR'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
    ichimoku = ta.trend.IchimokuIndicator(df['high'], df['low'], window1=9, window2=26, window3=52)
    df['Ichimoku_span_A'] = ichimoku.ichimoku_a()
    df['Ichimoku_span_B'] = ichimoku.ichimoku_b()
    df['ADX'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=14).adx()
    return df

# Generate signals
def generate_signals(df):
    signals = []
    for i in range(len(df)):
        if (
            df['RSI'][i] < 30 and
            df['close'][i] < df['EMA20'][i] and
            df['ADX'][i] > 25 and
            df['Ichimoku_span_A'][i] > df['Ichimoku_span_B'][i] and
            df['OBV'][i] > df['OBV'][i-1]
        ):
            signals.append({'time': df['timestamp'][i], 'action': 'BUY', 'price': df['close'][i]})
        elif (
            df['RSI'][i] > 70 and
            df['close'][i] > df['EMA20'][i] and
            df['ADX'][i] > 25 and
            df['Ichimoku_span_A'][i] < df['Ichimoku_span_B'][i] and
            df['OBV'][i] < df['OBV'][i-1]
        ):
            signals.append({'time': df['timestamp'][i], 'action': 'SELL', 'price': df['close'][i]})
    return signals

# Main Function
def analyze_assets(symbols):
    all_signals = []
    for symbol in symbols:
        try:
            print(f"Analyzing {symbol}...")
            data_15m = fetch_data(symbol, '15m')
            data_15m = calculate_indicators(data_15m)
            hurst = hurst_exponent(data_15m['close'])
            print(f"Hurst Exponent for {symbol}: {hurst:.2f}")
            if hurst > 0.5:
                print(f"{symbol} is trending; generating signals...")
                signals = generate_signals(data_15m)
                all_signals.extend(signals)
            else:
                print(f"{symbol} is mean-reverting; no signals generated.")
        except Exception as e:
            print(f"Error analyzing {symbol}: {e}")
    return all_signals

# Define symbols and run the strategy
symbols = ['BTC/USDT', 'ETH/USDT', 'NEAR/USDT', 'AVAX/USDT', 'SOL/USDT', 'DOGE/USDT', 'LTC/USDT', 'XAVA/USDT', 'SLIM/USDT' , 'HNT/USDT']
signals = analyze_assets(symbols)

# Output signals
if signals:
    print("Generated Signals:")
    for signal in signals:
        print(signal)
else:
    print("No signals generated.")


Analyzing BTC/USDT...
Hurst Exponent for BTC/USDT: 10.25
BTC/USDT is trending; generating signals...
Analyzing ETH/USDT...
Hurst Exponent for ETH/USDT: 10.26
ETH/USDT is trending; generating signals...
Analyzing NEAR/USDT...
Hurst Exponent for NEAR/USDT: 10.21
NEAR/USDT is trending; generating signals...
Analyzing AVAX/USDT...
Hurst Exponent for AVAX/USDT: 10.28
AVAX/USDT is trending; generating signals...
Analyzing SOL/USDT...
Hurst Exponent for SOL/USDT: 10.25
SOL/USDT is trending; generating signals...
Analyzing DOGE/USDT...
Hurst Exponent for DOGE/USDT: 10.26
DOGE/USDT is trending; generating signals...
Analyzing LTC/USDT...
Hurst Exponent for LTC/USDT: 10.34
LTC/USDT is trending; generating signals...
Analyzing XAVA/USDT...
Hurst Exponent for XAVA/USDT: 10.22
XAVA/USDT is trending; generating signals...
Analyzing SLIM/USDT...
Hurst Exponent for SLIM/USDT: 10.35
SLIM/USDT is trending; generating signals...
Analyzing HNT/USDT...
Hurst Exponent for HNT/USDT: 10.33
HNT/USDT is trendin